In [1]:
import pandas as pd
import tqdm
import numpy as np
import pandas as pd
from recommender.init_data.crawl_imdb import IMDbDataPydantic, wrapper, fetch_imdb_image, img_wrapper
import concurrent.futures
import tqdm
import time
import multiprocessing
import threading
from recommender import REPO_PATH
import os
os.chdir(REPO_PATH)
import requests
from bs4 import BeautifulSoup
import ast

In [10]:
df = pd.read_csv("data/imdb_data_images.csv", dtype = {"imdbId":str})

wanted_ids = df.loc[df.rating.notna() & df.images.isna(), "imdbId"].values

In [12]:
df.images.notna().sum()

9731

In [11]:
len(wanted_ids)

3

In [4]:
def fetch_imdb_data_with_timeout(imdbid, timeout = 20):
    # Create a multiprocessing Queue to get the function result
    queue = multiprocessing.Queue()

    process = multiprocessing.Process(target=img_wrapper, args=(queue, imdbid))
    process.start()

    process.join(timeout)
    if process.is_alive():
        # If the process is still alive after the timeout, terminate it
        process.terminate()
        process.join()
        return "Timeout reached"

    try:
        # Get the result from the queue
        result = queue.get_nowait()
    except multiprocessing.queues.Empty:
        return "No result returned"

    return result

data = fetch_imdb_data_with_timeout('1234567')  
data

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00\x84\x00\x06\x06\x06\x0b\x08\x0b\x0b\x0b\x0b\x0f\x19\x13\x13\x0f\x13\x13!\x18\x12\x18\x15\x1e\x13\x15\x15\x1e\x15\x1e\x1e\x15\x1b\x1e\x1b\x1b"\x1e\x19\x19\x1e"\x1b\x1e - %!(" + - -,,/34-\x1e3E\x01\x0b\x0b\x06\x0b\x0b\x0b\x0f\x0b\x0b\x19+\x1e\x13",%\x0f 5,4",/,,!:,4+,\x1b-,"((""% %,"//-\x13!/+\x1e>!"\x1e,+3(\x1e/8\xff\xc0\x00\x11\x08\x01\x19\x00\xbe\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\xa1\x00\x00\x01\x05\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x05\x01\x02\x03\x04\x06\x07\x00\x08\x10\x00\x01\x03\x02\x04\x03\x05\x04\x07\x07\x02\x05\x05\x00\x00\x00\x01\x00\x02\x03\x04\x11\x05\x12!1\x06AQ\x13"aq\x91\x142\x81\xa1\x07\x153BR\xc1\xd1\x16#Cr\x82\x92\xb1S\x83$D\x93\xe1\xf1%bc\xa2\xd2\x01\x00\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x11\x00\x02\x01\x03\x03\x04\x01\x03\x04\x01\x05\x00\x00\x00\x00\x00\x00\x01\x02\x03\x11\x12\x04\x13!\x141AQ\x05"2R

In [5]:
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    future_to_imdb_id = {executor.submit(fetch_imdb_data_with_timeout, imdb_id): imdb_id for imdb_id in wanted_ids}

    results = {}
    i = 0

    for future in tqdm.tqdm(concurrent.futures.as_completed(future_to_imdb_id), total=len(future_to_imdb_id)):
        imdb_id = future_to_imdb_id[future]
        try:
            data = future.result()
            results[imdb_id] = data
        except concurrent.futures.TimeoutError:
            print(f"Fetching data for IMDb ID {imdb_id} timed out.")

  0%|          | 0/104 [00:00<?, ?it/s]

 10%|▉         | 10/104 [00:09<01:14,  1.25it/s]Process Process-13:
Traceback (most recent call last):
  File "/Users/fhammer/miniconda3/envs/reco/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/fhammer/miniconda3/envs/reco/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/fhammer/Documents/ai_and_the_web/recomender/recommender/init_data/crawl_imdb.py", line 149, in img_wrapper
    result = fetch_imdb_image(args, timeout=5)
  File "/Users/fhammer/Documents/ai_and_the_web/recomender/recommender/init_data/crawl_imdb.py", line 45, in fetch_imdb_image
    raise ("did not find hero media")
TypeError: exceptions must derive from BaseException
 97%|█████████▋| 101/104 [01:14<00:01,  1.70it/s]Process Process-105:
Traceback (most recent call last):
  File "/Users/fhammer/miniconda3/envs/reco/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
 

In [6]:
isinstance(list(results.values())[0], bytes)

True

In [9]:
# df = df.set_index("imdbId").copy()
for i,v in results.items():
    
    if not isinstance(v, bytes):
        continue
    
    df.loc[i, "images"] = v
    
    
df.to_csv("data/imdb_data_images.csv")

In [37]:
for name, image in df.query("images.notna()").set_index("imdbId").images.items():
    
    
    with open(REPO_PATH / f"recommender/static/images/{name}.jpg","wb") as f:
        image = ast.literal_eval(image)
        f.write(image)